In [1]:
!pip install pyspark
!pip install imbalanced-learn


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("DiabetesBigDataAnalytics") \
    .getOrCreate()


In [5]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv -O diabetes.csv


--2026-01-16 14:18:05--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘diabetes.csv’

diabetes.csv        100%[===================>]  22.73K  --.-KB/s    in 0.001s  

2026-01-16 14:18:05 (16.3 MB/s) - ‘diabetes.csv’ saved [23278/23278]



In [6]:
df = spark.read.csv("diabetes.csv", inferSchema=True)


In [7]:
df = df.toDF(
    "Pregnancies","Glucose","BloodPressure",
    "SkinThickness","Insulin","BMI",
    "DiabetesPedigreeFunction","Age","Outcome"
)


In [8]:
df.show(5)
df.printSchema()


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows
root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (null

In [9]:
df.printSchema()
df.describe().show()
df.groupBy("Outcome").count().show()


root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+------------------+
|  count|               768|              768|       

In [10]:
from pyspark.sql.functions import col, when

for c in df.columns:
    median = df.approxQuantile(c, [0.5], 0.01)[0]
    df = df.withColumn(c, when(col(c).isNull(), median).otherwise(col(c)))


In [11]:
zero_cols = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]

for c in zero_cols:
    median = df.approxQuantile(c, [0.5], 0.01)[0]
    df = df.withColumn(c, when(col(c)==0, median).otherwise(col(c)))


In [12]:
def cap_outliers(df, c):
    q1 = df.approxQuantile(c, [0.25], 0.01)[0]
    q3 = df.approxQuantile(c, [0.75], 0.01)[0]
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return df.withColumn(
        c,
        when(col(c)<lower, lower)
        .when(col(c)>upper, upper)
        .otherwise(col(c))
    )

for c in ["Glucose","BloodPressure","Insulin","BMI"]:
    df = cap_outliers(df, c)


In [13]:
pdf = df.toPandas()

from imblearn.over_sampling import SMOTE
X = pdf.drop("Outcome", axis=1)
y = pdf["Outcome"]

X_bal, y_bal = SMOTE().fit_resample(X, y)

import pandas as pd
df = spark.createDataFrame(pd.concat([X_bal, y_bal], axis=1))


In [14]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(
    inputCols=X.columns.tolist(),
    outputCol="raw_features"
)

df_vec = assembler.transform(df)

scaler = StandardScaler(
    inputCol="raw_features",
    outputCol="features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(df_vec)
final_df = scaler_model.transform(df_vec)


In [15]:
train, test = final_df.randomSplit([0.8, 0.2], seed=42)


In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [17]:
accuracy_eval = MulticlassClassificationEvaluator(
    labelCol="Outcome",
    predictionCol="prediction",
    metricName="accuracy"
)

precision_eval = MulticlassClassificationEvaluator(
    labelCol="Outcome",
    predictionCol="prediction",
    metricName="weightedPrecision"
)

recall_eval = MulticlassClassificationEvaluator(
    labelCol="Outcome",
    predictionCol="prediction",
    metricName="weightedRecall"
)

f1_eval = MulticlassClassificationEvaluator(
    labelCol="Outcome",
    predictionCol="prediction",
    metricName="f1"
)


In [20]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

# Define the models
models = {
    "Logistic Regression": LogisticRegression(labelCol="Outcome", featuresCol="features"),
    "Decision Tree": DecisionTreeClassifier(labelCol="Outcome", featuresCol="features"),
    "Random Forest": RandomForestClassifier(labelCol="Outcome", featuresCol="features"),
    "GBTClassifier": GBTClassifier(labelCol="Outcome", featuresCol="features")
}

cresults = []

for name, model in models.items():
    trained_model = model.fit(train)
    predictions = trained_model.transform(test)

    accuracy = accuracy_eval.evaluate(predictions)
    precision = precision_eval.evaluate(predictions)
    recall = recall_eval.evaluate(predictions)
    f1 = f1_eval.evaluate(predictions)

    cresults.append((
        name,
        round(accuracy, 4),
        round(precision, 4),
        round(recall, 4),
        round(f1, 4)
    ))

    print(f"\n{name}")
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-Score : {f1:.4f}")


Logistic Regression
Accuracy : 0.7308
Precision: 0.7318
Recall   : 0.7308
F1-Score : 0.7311

Decision Tree
Accuracy : 0.7637
Precision: 0.7716
Recall   : 0.7637
F1-Score : 0.7588

Random Forest
Accuracy : 0.7912
Precision: 0.7928
Recall   : 0.7912
F1-Score : 0.7895

GBTClassifier
Accuracy : 0.7692
Precision: 0.7703
Recall   : 0.7692
F1-Score : 0.7674


In [21]:
results_df = spark.createDataFrame(
    cresults,
    ["Model", "Accuracy", "Precision", "Recall", "F1-Score"]
)

results_df.show(truncate=False)

+-------------------+--------+---------+------+--------+
|Model              |Accuracy|Precision|Recall|F1-Score|
+-------------------+--------+---------+------+--------+
|Logistic Regression|0.7308  |0.7318   |0.7308|0.7311  |
|Decision Tree      |0.7637  |0.7716   |0.7637|0.7588  |
|Random Forest      |0.7912  |0.7928   |0.7912|0.7895  |
|GBTClassifier      |0.7692  |0.7703   |0.7692|0.7674  |
+-------------------+--------+---------+------+--------+



In [22]:
best_model = results_df.orderBy("F1-Score", ascending=False).first()
print("Best Model:", best_model["Model"])

Best Model: Random Forest


In [23]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

gbt = GBTClassifier(labelCol="Outcome")

paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [3, 5, 7]) \
    .addGrid(gbt.maxIter, [20, 50]) \
    .build()

evaluator = BinaryClassificationEvaluator(
    labelCol="Outcome",
    metricName="areaUnderROC"
)

cv = CrossValidator(
    estimator=gbt,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3
)

cv_model = cv.fit(train)
best_model = cv_model.bestModel


In [24]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# 1️⃣ Define the model
gbt = GBTClassifier(labelCol="Outcome")

# 2️⃣ Define a smaller hyperparameter grid for speed
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [3, 5]) \
    .addGrid(gbt.maxIter, [20]) \
    .build()

# 3️⃣ Binary evaluator for TrainValidationSplit
evaluator = BinaryClassificationEvaluator(labelCol="Outcome", metricName="areaUnderROC")

# 4️⃣ TrainValidationSplit (much faster than CV)
tvs = TrainValidationSplit(
    estimator=gbt,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,  # 80% train, 20% validation
    parallelism=2    # adjust based on CPU cores
)

# 5️⃣ Fit the model
tvs_model = tvs.fit(train)

# 6️⃣ Get best model
best_model = tvs_model.bestModel

# 7️⃣ Make predictions on test set
predictions = best_model.transform(test)

# 8️⃣ Evaluators for metrics
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="accuracy"
)
precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="precisionByLabel", metricLabel=1.0
)
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="recallByLabel", metricLabel=1.0
)
f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="f1"
)

# 9️⃣ Calculate metrics
accuracy = accuracy_evaluator.evaluate(predictions)
precision = precision_evaluator.evaluate(predictions)
recall = recall_evaluator.evaluate(predictions)
f1 = f1_evaluator.evaluate(predictions)

# 10️⃣ Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# 11️⃣ Optional: best hyperparameters
print("Best MaxDepth:", best_model.getOrDefault("maxDepth"))
print("Best MaxIter:", best_model.getOrDefault("maxIter"))


Accuracy: 0.7802
Precision: 0.7810
Recall: 0.8283
F1 Score: 0.7793
Best MaxDepth: 3
Best MaxIter: 20


In [25]:
pred = best_model.transform(test)

print("Accuracy:", accuracy_eval.evaluate(pred))
print("F1 Score:", f1_eval.evaluate(pred))


Accuracy: 0.7802197802197802
F1 Score: 0.7793309631544927


In [26]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# =======================
# 1️⃣ Define the model
# =======================
gbt = GBTClassifier(labelCol="Outcome")

# =======================
# 2️⃣ Hyperparameter grid (fast version)
# =======================
paramGrid = (
    ParamGridBuilder()
    .addGrid(gbt.maxDepth, [3, 5])       # test 2 depths
    .addGrid(gbt.maxIter, [20, 50])      # test 2 iterations
    .addGrid(gbt.stepSize, [0.1])        # fixed step size
    .build()
)

# =======================
# 3️⃣ Binary evaluator for TrainValidationSplit
# =======================
evaluator = BinaryClassificationEvaluator(
    labelCol="Outcome",
    metricName="areaUnderROC"
)

# =======================
# 4️⃣ TrainValidationSplit
# =======================
tvs = TrainValidationSplit(
    estimator=gbt,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,      # 80% train, 20% validation
    parallelism=2         # adjust based on CPU cores
)

# =======================
# 5️⃣ Fit the model
# =======================
tvs_model = tvs.fit(train)

# =======================
# 6️⃣ Get the best model
# =======================
best_model = tvs_model.bestModel

# =======================
# 7️⃣ Make predictions on test set
# =======================
predictions = best_model.transform(test)

# =======================
# 8️⃣ Evaluate metrics
# =======================
accuracy_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="accuracy"
)
precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="precisionByLabel", metricLabel=1.0
)
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="recallByLabel", metricLabel=1.0
)
f1_evaluator = MulticlassClassificationEvaluator(
    labelCol="Outcome", predictionCol="prediction", metricName="f1"
)

accuracy = accuracy_evaluator.evaluate(predictions)
precision = precision_evaluator.evaluate(predictions)
recall = recall_evaluator.evaluate(predictions)
f1 = f1_evaluator.evaluate(predictions)

# =======================
# 9️⃣ Print metrics
# =======================
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# =======================
# 🔟 Print best hyperparameters
# =======================
print("Best MaxDepth:", best_model.getOrDefault("maxDepth"))
print("Best MaxIter:", best_model.getOrDefault("maxIter"))
print("Best StepSize:", best_model.getOrDefault("stepSize"))

# =======================
# 1️⃣1️⃣ Optional: Save the model
# =======================
# best_model.save("best_gbt_model")


Accuracy: 0.7967
Precision: 0.7818
Recall: 0.8687
F1 Score: 0.7948
Best MaxDepth: 3
Best MaxIter: 50
Best StepSize: 0.1


to improve accuracy

In [27]:
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import when


In [28]:
# Calculate class weights
total = train.count()
pos = train.filter("Outcome==1").count()
neg = total - pos

weight_for_1 = total / (2 * pos)
weight_for_0 = total / (2 * neg)

# Add weight column
train = train.withColumn("classWeight", when(train["Outcome"]==1, weight_for_1).otherwise(weight_for_0))


In [29]:
# GBT
gbt = GBTClassifier(labelCol="Outcome", weightCol="classWeight")

# Random Forest
rf = RandomForestClassifier(labelCol="Outcome", weightCol="classWeight")

# Logistic Regression
lr = LogisticRegression(labelCol="Outcome", weightCol="classWeight")


In [30]:
# GBT Grid
gbt_paramGrid = (
    ParamGridBuilder()
    .addGrid(gbt.maxDepth, [3, 5, 7])
    .addGrid(gbt.maxIter, [50, 100])
    .addGrid(gbt.stepSize, [0.05, 0.1])
    .build()
)

# Random Forest Grid
rf_paramGrid = (
    ParamGridBuilder()
    .addGrid(rf.numTrees, [50, 100])
    .addGrid(rf.maxDepth, [5, 7])
    .build()
)

# Logistic Regression Grid
lr_paramGrid = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.1])
    .addGrid(lr.elasticNetParam, [0.0, 0.5])
    .build()
)


In [31]:
evaluator = BinaryClassificationEvaluator(labelCol="Outcome", metricName="areaUnderROC")


In [32]:
def train_model(estimator, paramGrid, train):
    tvs = TrainValidationSplit(
        estimator=estimator,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        trainRatio=0.8,
        parallelism=2  # adjust based on CPU cores
    )
    model = tvs.fit(train)
    return model.bestModel


In [33]:
best_gbt = train_model(gbt, gbt_paramGrid, train)
best_rf = train_model(rf, rf_paramGrid, train)
best_lr = train_model(lr, lr_paramGrid, train)


In [34]:
predictions_gbt = best_gbt.transform(test)
predictions_rf = best_rf.transform(test)
predictions_lr = best_lr.transform(test)


In [35]:
def evaluate(pred):
    acc = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy").evaluate(pred)
    precision = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="precisionByLabel", metricLabel=1.0).evaluate(pred)
    recall = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="recallByLabel", metricLabel=1.0).evaluate(pred)
    f1 = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="f1").evaluate(pred)
    return acc, precision, recall, f1

metrics_gbt = evaluate(predictions_gbt)
metrics_rf = evaluate(predictions_rf)
metrics_lr = evaluate(predictions_lr)

print("GBT Metrics: Accuracy={:.4f}, Precision={:.4f}, Recall={:.4f}, F1={:.4f}".format(*metrics_gbt))
print("RF Metrics: Accuracy={:.4f}, Precision={:.4f}, Recall={:.4f}, F1={:.4f}".format(*metrics_rf))
print("LR Metrics: Accuracy={:.4f}, Precision={:.4f}, Recall={:.4f}, F1={:.4f}".format(*metrics_lr))


GBT Metrics: Accuracy=0.8132, Precision=0.7876, Recall=0.8990, F1=0.8108
RF Metrics: Accuracy=0.8077, Precision=0.7909, Recall=0.8788, F1=0.8059
LR Metrics: Accuracy=0.7253, Precision=0.7579, Recall=0.7273, F1=0.7257


In [36]:
best_model_name = max(
    [("GBT", metrics_gbt[0], best_gbt),
     ("RF", metrics_rf[0], best_rf),
     ("LR", metrics_lr[0], best_lr)],
    key=lambda x: x[1]
)

print(f"Best model: {best_model_name[0]} with Accuracy: {best_model_name[1]:.4f}")
best_model_to_use = best_model_name[2]


Best model: GBT with Accuracy: 0.8132


In [ ]:
to improve beyond 0.85

In [37]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import when


In [38]:
# Compute class weights
total = train.count()
pos = train.filter("Outcome==1").count()
neg = total - pos

weight_for_1 = total / (2 * pos)
weight_for_0 = total / (2 * neg)

# Add class weight column
train = train.withColumn("classWeight", when(train["Outcome"]==1, weight_for_1).otherwise(weight_for_0))


In [39]:
# Random Forest (main model)
rf = RandomForestClassifier(labelCol="Outcome", weightCol="classWeight", seed=42)

# GBT (optional for ensemble)
gbt = GBTClassifier(labelCol="Outcome", weightCol="classWeight", seed=42)


In [40]:
# RF grid
rf_paramGrid = (
    ParamGridBuilder()
    .addGrid(rf.numTrees, [100, 150])
    .addGrid(rf.maxDepth, [5, 7, 10])
    .addGrid(rf.subsamplingRate, [0.8, 1.0])
    .build()
)

# GBT grid (optional)
gbt_paramGrid = (
    ParamGridBuilder()
    .addGrid(gbt.maxDepth, [3, 5, 7])
    .addGrid(gbt.maxIter, [50, 100])
    .addGrid(gbt.stepSize, [0.05, 0.1])
    .build()
)


In [41]:
evaluator = BinaryClassificationEvaluator(labelCol="Outcome", metricName="areaUnderROC")


In [42]:
def train_model(estimator, paramGrid, train_df):
    tvs = TrainValidationSplit(
        estimator=estimator,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        trainRatio=0.8,
        parallelism=4  # use all CPU cores
    )
    best_model = tvs.fit(train_df).bestModel
    return best_model


In [43]:
best_rf = train_model(rf, rf_paramGrid, train)
best_gbt = train_model(gbt, gbt_paramGrid, train)  # optional for ensemble


In [44]:
test.cache()  # cache for faster repeated access

pred_rf = best_rf.transform(test)
pred_gbt = best_gbt.transform(test)


In [45]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

# Average probabilities of class 1
avg_prob_udf = udf(lambda rf_prob, gbt_prob: (rf_prob[1] + gbt_prob[1])/2, DoubleType())

ensemble_df = pred_rf.alias("rf").join(
    pred_gbt.select(col("Outcome"), col("probability").alias("gbt_probability")).alias("gbt"),
    on="Outcome",
    how="inner"
) \
.withColumn("avg_prob", avg_prob_udf(col("rf.probability"), col("gbt.gbt_probability"))) \
.withColumn("prediction", (col("avg_prob") > 0.5).cast("double"))

In [46]:
def evaluate(pred_df):
    acc = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="accuracy").evaluate(pred_df)
    precision = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="precisionByLabel", metricLabel=1.0).evaluate(pred_df)
    recall = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="recallByLabel", metricLabel=1.0).evaluate(pred_df)
    f1 = MulticlassClassificationEvaluator(labelCol="Outcome", predictionCol="prediction", metricName="f1").evaluate(pred_df)
    return acc, precision, recall, f1

accuracy, precision, recall, f1 = evaluate(pred_rf)  # or ensemble_df for ensemble
print(f"Final Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


Final Accuracy: 0.8297
Precision: 0.8036, Recall: 0.9091, F1: 0.8277


In [50]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("diabetes.csv", header=None)

# Define columns for the dataset
column_names = [
    "Pregnancies", "Glucose", "BloodPressure",
    "SkinThickness", "Insulin", "BMI",
    "DiabetesPedigreeFunction", "Age", "Outcome"
]
df.columns = column_names

# Define features and target
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train Random Forest
best_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
best_model.fit(X_train, y_train)

# Evaluate
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {acc:.4f}")

Model Accuracy: 0.7597


In [51]:
with open("best_diabetes_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Best model saved as best_diabetes_model.pkl")


Best model saved as best_diabetes_model.pkl
